# Task 1: Generating ML-KEM Key Pairs

## Learning Objectives

Upon completing this task, you should be able to:

1.  **Understand the concept** of public-key and private-key pair generation in the context of Post-Quantum Cryptography (PQC).
2.  **Identify** ML-KEM (Module-Lattice-based Key-Establishment Mechanism, based on CRYSTALS-Kyber) as a NIST-standardized PQC algorithm for key encapsulation.
3.  **Interact** with a Python script using `liboqs` to generate ML-KEM key pairs for different security levels (e.g., ML-KEM-512, ML-KEM-768, ML-KEM-1024).
4.  **Distinguish** between a public key (sharable) and a private key (secret).
5.  **Recognize** that the generated keys are binary data, often represented in Base64 encoding for easier display and handling.
6.  **Appreciate** that different security levels result in different key sizes and other parameter variations.

## Understanding the Task and the Code

In this task, we'll use a Python script to generate key pairs for the ML-KEM algorithm. ML-KEM is designed to be secure against attacks from both classical and (anticipated) quantum computers. It's primarily used for **Key Encapsulation Mechanisms (KEMs)**, where its main job is to help two parties securely agree on a shared secret key, which can then be used for symmetric encryption.

**When you click the "Generate Keys" button:**

*   It gets the selected ML-KEM algorithm from the dropdown.
*   It uses `oqs.KeyEncapsulation(selected_kem_alg)` to create a KEM object configured for the chosen algorithm.
*   `public_key = kem.generate_keypair()`: This is the core function call that generates both the public key and the private key. The public key is returned directly.
*   `secret_key = kem.export_secret_key()`: This retrieves the corresponding private key from the KEM object.
*   The binary public and private keys are then encoded into Base64 strings (which use only ASCII characters) and displayed in the text areas. Base64 is used here to make it easy for you to see and manually copy the keys.
*   Details about the algorithm and key lengths are printed to the status area.

**Your Interaction:**

*   Run the Python code cell below.
*   Select an "ML-KEM Algorithm" from the dropdown.
*   Click the "Generate Keys" button.
*   Observe the generated Base64 encoded public and private keys.
*   Read the details printed in the status area. Try generating keys for different algorithm variants to see how the details (like key lengths) change.

## Next Steps

The public and private keys you generate here (specifically, their Base64 string representations) will be crucial for the subsequent tasks:
*   **Task 2 (Encryption/Encapsulation):** You will use the **public key** to protect a shared secret.
*   **Task 3 (Decryption/Decapsulation):** You will use the **private key** to recover that shared secret.

Make sure you understand which key is which and how to copy them accurately!

---
**Now, run the code cell below to perform Task 1.**

In [11]:
import oqs
import ipywidgets as widgets
from IPython.display import display, HTML as IPHTML # Alias IPHTML for clarity when used outside VBox
import base64

# --- Available Kyber Algorithms from liboqs (using NIST Standard Names) ---
supported_kems = ['ML-KEM-512', 'ML-KEM-768', 'ML-KEM-1024']

# --- UI Elements ---
header = IPHTML("<h2>Task 1: Generate Kyber Key Pair (ML-KEM)</h2>")
description = IPHTML("""
<p>This task demonstrates the generation of a public and private key pair using a Post-Quantum Key Encapsulation Mechanism (KEM), specifically <b>ML-KEM</b> (based on CRYSTALS-Kyber), as standardized in FIPS 203.</p>
<p>Select an ML-KEM algorithm variant (e.g., ML-KEM-768 for NIST Security Level 3) and click 'Generate Keys'. The public key can be shared, while the private key must be kept secret by the intended recipient of encrypted messages.</p>
<p>The generated keys will appear below as Base64 encoded strings. You will manually copy these for use in Task 2 and Task 3.</p>
""")

kem_dropdown = widgets.Dropdown(
    options=supported_kems,
    value=supported_kems[1] if len(supported_kems) > 1 else (supported_kems[0] if supported_kems else None), # Default to ML-KEM-768
    description='ML-KEM Algorithm:',
    disabled=not supported_kems,
    style={'description_width': 'initial'}
)

generate_button = widgets.Button(
    description='Generate Keys',
    button_style='success',
    tooltip='Click to generate ML-KEM public and private keys',
    icon='key'
)

public_key_output = widgets.Textarea(
    value='',
    placeholder='Public Key will appear here (Base64 encoded). Copy this for Task 2.',
    description='Public Key:',
    layout={'width': '95%', 'height': '100px'},
    disabled=True,
    style={'description_width': 'initial'}
)

private_key_output = widgets.Textarea(
    value='',
    placeholder='Private Key will appear here (Base64 encoded). Copy this for Task 3.',
    description='Private Key:',
    layout={'width': '95%', 'height': '100px'},
    disabled=True,
    style={'description_width': 'initial'}
)

status_output = widgets.Output(layout={'width': '95%'})

# UPDATED save_instructions
next_steps_info = widgets.HTML(value="""
    <hr>
    <h4>Next Steps:</h4>
    <p>The <b>Base64 encoded keys</b> displayed above will be used in the next tasks:</p>
    <ul>
        <li>The <b>Public Key</b> (from the first text area) will be manually copied and pasted into <b>Task 2 (Encryption/Encapsulation)</b> by the sender to encapsulate a shared secret.</li>
        <li>The <b>Private Key</b> (from the second text area) will be manually copied and pasted into <b>Task 3 (Decryption/Decapsulation)</b> by the recipient to decapsulate the shared secret.</li>
    </ul>
    <p>Make sure to copy the <em>entire</em> Base64 string accurately for each key.</p>
    """,
    layout={'width': '95%'}
)

# Store actual key bytes globally for potential internal access if needed,
# though the lab flow now emphasizes copy-pasting the Base64 strings.
_generated_keys_bytes = {
    "public": None,
    "private": None,
    "algorithm": None
}

# --- Button Click Handler ---
def on_generate_button_clicked(b):
    selected_kem_alg = kem_dropdown.value
    public_key_output.value = ""
    private_key_output.value = ""
    _generated_keys_bytes["public"] = None
    _generated_keys_bytes["private"] = None
    _generated_keys_bytes["algorithm"] = None

    with status_output:
        status_output.clear_output()
        if not selected_kem_alg:
            print("Error: No KEM algorithm selected. Please choose an algorithm from the dropdown.")
            return

        print(f"Generating keys for {selected_kem_alg}...")
        try:
            with oqs.KeyEncapsulation(selected_kem_alg) as kem:
                public_key = kem.generate_keypair()
                secret_key = kem.export_secret_key()

                _generated_keys_bytes["public"] = public_key
                _generated_keys_bytes["private"] = secret_key
                _generated_keys_bytes["algorithm"] = selected_kem_alg

                public_key_b64 = base64.b64encode(public_key).decode('utf-8')
                private_key_b64 = base64.b64encode(secret_key).decode('utf-8')

                public_key_output.value = public_key_b64
                private_key_output.value = private_key_b64
                
                print(f"Successfully generated key pair for {selected_kem_alg}.")
                print(f"You can now copy the Base64 encoded Public Key and Private Key from the text areas above for the next tasks.")
                print(f"\nDetails from liboqs:")
                print(f"  Algorithm: {kem.details['name']}")
                print(f"  Claimed NIST security level: {kem.details['claimed_nist_level']}")
                print(f"  Is IND-CCA secure: {kem.details['is_ind_cca']}")
                print(f"  Public Key Length (bytes): {kem.details['length_public_key']}")
                print(f"  Private Key Length (bytes): {kem.details['length_secret_key']}")
                print(f"  Ciphertext Length (bytes for encapsulated key): {kem.details['length_ciphertext']}")

        except oqs.MechanismNotSupportedError:
            print(f"Error: The KEM algorithm '{selected_kem_alg}' is not supported by your liboqs build.")
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            import traceback
            traceback.print_exc()

generate_button.on_click(on_generate_button_clicked)

# --- Display UI ---
display(header)
display(description)

main_ui_container = widgets.VBox([
    kem_dropdown,
    generate_button,
    public_key_output,
    private_key_output,
    status_output,
    next_steps_info # Changed variable name for clarity
], layout={'width': '100%'})

display(main_ui_container)